In [ ]:
import numpy as np
import pandas as pd
import struct
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
npxor = np.frompyfunc(np.bitwise_xor, 2, 1)

In [ ]:
def lzc(t):
    return 32 - len(np.binary_repr(t))
lzcv = np.frompyfunc(lzc, 1, 1)

In [ ]:
def to_u32(f):
    s = struct.pack('>f',f)
    return struct.unpack('>l', s)[0]
to_u32v = np.frompyfunc(to_u32, 1, 1)

In [ ]:
def mean_lzc(m, o):
    dist = np.random.normal(m,o,size=1000)
    truth = to_u32(m)
    vals = to_u32v(dist)
    xors = npxor(vals, truth)
    lzcs = lzcv(xors)
    return lzcs.mean()

def mean_lzc_group(m, o):
    dist = np.random.normal(m,o,size=1000)
    truth = to_u32(m)
    vals = to_u32v(dist)
    xors = npxor(vals, truth)
    lzcs = lzcv(xors)
    return lzcs
    return lzcs.mean()

## TODO

- [x] Plot LZC($\mu$)
- [x] Plot LZC($\sigma$)
- [ ] Find out with what $\sigma$ we are working for different predictors for climate data
    - [ ] Mean/Std of residuals?
    - [ ] Mean/Std of LZC of residuals?
    - [ ] Mean/Std of Error of predictor?
- [ ] Plot compression ratio dependency of $\mu$
- [ ] Plot compression ratio dependency of $\sigma$
- [x] Mark the 0101010101 shift values in the plot
- [x] Mark the 1010101010 shift values in the plot
- [x] Change the binary representation of numbers to grey plot LZC

# Calculate shift goals

In [ ]:
def get_flipping_value(val):
    assert val & (val-1) == 0, "Not power of two"
    i = len(np.binary_repr(val)) - 3
    while i >= 0:
        val += 1 << i
        i -= 2
    return val

def get_mirrored_flipping_value(val):
#     return int(get_flipping_value(val) - 2*(get_flipping_value(val) - val*1.25))
    flipped = get_flipping_value(val)
    b = val + (val >> 2)
    a = (flipped - b) << 1
    return flipped - a - 1 

def get_close_flipping_value(val):
    # 2**x + ((mflip - 2**x) >> 2)
    return val + ((get_mirrored_flipping_value(val) - val) >> 2)

In [ ]:
def get_closest_pow_2(val):
    next_smaller = next_smaller_pow_two(val)
    diff_smaller = val - next_smaller
    next_bigger = next_bigger_pow_two(val)
    diff_bigger = next_bigger - val
    if diff_bigger < diff_smaller:
        return next_bigger
    else:
        return next_smaller

def next_bigger_pow_two(val):
    return 1 << len(np.binary_repr(val))

def next_smaller_pow_two(val):
    return 1 << len(np.binary_repr(val)) - 1

In [ ]:
def get_shifted_goal(val, method='closest', mode='flip'):
    # Calculate power of two on which the shift should be based
    if val != 0 and val & (val-1) == 0:
        val = val
    elif method in ('closest', 'c'):
        val = get_closest_pow_2(val)
    elif method in ('smaller', 's'):
        val = next_smaller_pow_two(val)
    elif method in ('bigger', 'b'):
        val = next_bigger_pow_two(val)
    else:
        raise Exception("Could not understand method '{}'".format(method))
    
    if mode == 'flip':
        return get_flipping_value(val)
    elif mode == 'mflip':
        return get_mirrored_flipping_value(val)
    elif mode == 'cflip':
        return get_close_flipping_value(val)
    else:
        raise Exception("Could not understand mode '{}'".format(mode))    

# LZC($\mu$)

In [ ]:
mu = np.arange(1000)
sigma1 =  64 * mu
sigma2 = 128 * mu
sigma3 = 256 * mu
sigma4 = 512 * mu

result1_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma1)]
result2_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma2)]
result3_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma3)]
result4_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma4)]

factors = [1.75,1.625,1.5,1.25,1.125,1]
fcolors = ['crimson','magenta','skyblue','limegreen','olivedrab', 'goldenrod']

# LZC(mu)
_, ax = plt.subplots(figsize=(15,5))
sns.lineplot(data=pd.DataFrame({'$\sigma = 64 \cdot \mu$':result1_x_is_mu, '$\sigma = 128 \cdot \mu$':result2_x_is_mu, 
                                '$\sigma = 256 \cdot \mu$':result3_x_is_mu, '$\sigma = 512 \cdot \mu$':result4_x_is_mu})[1:])
plt.legend()
plt.title("LZC($\mu$)")
ax.set_xlabel("$\mu$ of random normal distribution")
ax.set_ylabel("Average LZC count of distribution")
# plt.savefig('lzc_mu_set2.svg')
plt.show()

In [ ]:
mu = np.arange(1000)
sigma1 = [1] * mu.size
sigma2 = np.sqrt(mu)
sigma3 = 1./mu
sigma4 = [10] * mu

result1_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma1)]
result2_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma2)]
result3_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma3)]
result4_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma4)]

factors = [1.75,1.625,1.5,1.25,1.125,1]
fcolors = ['crimson','magenta','skyblue','limegreen','olivedrab', 'goldenrod']

# LZC(mu)
_, ax = plt.subplots(figsize=(15,5))
sns.lineplot(data=pd.DataFrame({'$\sigma = 1$':result1_x_is_mu, '$\sigma = \sqrt{\mu}$':result2_x_is_mu, 
                                '$\sigma = \mu^{-1}$':result3_x_is_mu, '$\sigma = 10$':result4_x_is_mu})[1:])
# for x in range(4,10):
#     flip = get_shifted_goal(2**x, method='c', mode='flip')
#     mflip = get_shifted_goal(2**x, method='c', mode='mflip')
#     cflip = get_shifted_goal(2**x, method='c', mode='cflip')
#     _ = plt.axvline(2**x, alpha=.5, color=fcolors[x-4], ls=":")
#     _ = plt.axvline(flip, alpha=.5, color=fcolors[x-4], ls="-")
#     _ = plt.axvline(mflip, alpha=.5, color=fcolors[x-4], ls="-")
#     _ = plt.axvline(cflip, alpha=.5, color=fcolors[x-4], ls="-")
    
plt.legend()
plt.title("LZC($\mu$)")
ax.set_xlabel("$\mu$ of random normal distribution")
ax.set_ylabel("Average LZC count of distribution")
# plt.savefig('lzc_mu_set2.svg')
plt.show()

In [ ]:
# LZC(mu)
_, ax = plt.subplots(figsize=(15,5))
sns.lineplot(data=pd.DataFrame({'$\sigma = 1$':result1_x_is_mu, '$\sigma = \sqrt{\mu}$':result2_x_is_mu, 
                                '$\sigma = \mu^{-1}$':result3_x_is_mu, '$\sigma = 10$':result4_x_is_mu})[1:])
# for x in range(4,10):
#     flip = get_shifted_goal(2**x, method='c', mode='flip')
#     mflip = get_shifted_goal(2**x, method='c', mode='mflip')
#     cflip = get_shifted_goal(2**x, method='c', mode='cflip')
#     _ = plt.axvline(2**x, alpha=.5, color=fcolors[x-4], ls=":")
#     _ = plt.axvline(flip, alpha=.5, color=fcolors[x-4], ls="-")
#     _ = plt.axvline(mflip, alpha=.5, color=fcolors[x-4], ls="-")
#     _ = plt.axvline(cflip, alpha=.5, color=fcolors[x-4], ls="-")
plt.xlim((480, 700))
plt.legend()
plt.title("LZC($\mu$) for $\mu \in [480;700)$")
ax.set_xlabel("$\mu$ of random normal distribution")
ax.set_ylabel("Average LZC count of distribution")
# plt.savefig('lzc_mu_set2_zoom.svg')
plt.show()

In [ ]:
mu = np.arange(1000)
sigma1 = mu*.20
sigma2 = mu*.10
sigma3 = mu*.05
sigma4 = mu*.01

result1_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma1)]
result2_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma2)]
result3_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma3)]
result4_x_is_mu = [mean_lzc(m,o) for m,o in zip(mu,sigma4)]

factors = [1.75,1.625,1.5,1.25,1.125,1]
fcolors = ['crimson','magenta','skyblue','limegreen','olivedrab', 'goldenrod']

# LZC(mu)
_, ax = plt.subplots(figsize=(15,5))
sns.lineplot(data=pd.DataFrame({'$\sigma = 0.20 \cdot \mu$':result1_x_is_mu, '$\sigma = 0.10 \cdot \mu$':result2_x_is_mu, 
                                '$\sigma = 0.05 \cdot \mu$':result3_x_is_mu, '$\sigma = 0.01 \cdot \mu$':result4_x_is_mu})[1:])
# for i,f in enumerate(factors):
#     [plt.axvline(2**x*f, alpha=.5, color=fcolors[i], ls=":") for x in range(4,10)];
plt.legend()
plt.title("LZC($\mu$)")
ax.set_xlabel("$\mu$ of random normal distribution")
ax.set_ylabel("Average LZC count of distribution")
# plt.savefig('lzc_mu_set2.svg')
plt.show()

In [ ]:
# Vertical lines at above plot
base = 128
for i,f in enumerate(factors):
    print("{:.4f} {:>9} {:>10}".format(f, np.binary_repr(int(base*f), 8), fcolors[i]))

# LZC($\mu$) with shift goals

In [ ]:
# LZC(mu)
_, ax = plt.subplots(figsize=(15,5))
sns.lineplot(data=pd.DataFrame({'$\sigma = 0.20 \cdot \mu$':result1_x_is_mu, '$\sigma = 0.10 \cdot \mu$':result2_x_is_mu, 
                                '$\sigma = 0.05 \cdot \mu$':result3_x_is_mu, '$\sigma = 0.01 \cdot \mu$':result4_x_is_mu})[1:])
for x in range(4,10):
    flip = get_shifted_goal(2**x, method='c', mode='flip')
    mflip = get_shifted_goal(2**x, method='c', mode='mflip')
    cflip = get_shifted_goal(2**x, method='c', mode='cflip')
    _ = plt.axvline(2**x, alpha=.5, color=fcolors[x-4], ls=":")
    _ = plt.axvline(flip, alpha=.5, color=fcolors[x-4], ls="-")
    _ = plt.axvline(mflip, alpha=.5, color=fcolors[x-4], ls="-")
    _ = plt.axvline(cflip, alpha=.5, color=fcolors[x-4], ls="-")
plt.legend()
plt.title("LZC($\mu$)")
ax.set_xlabel("$\mu$ of random normal distribution")
ax.set_ylabel("Average LZC count of distribution")
# plt.savefig('lzc_mu.svg')
plt.show()

In [ ]:
x = 512
(f,mf,cf) = (get_shifted_goal(x, method='c', mode='flip'), 
             get_shifted_goal(x, method='c', mode='mflip'),
             get_shifted_goal(x, method='c', mode='cflip')
            )

In [ ]:
[(r, np.binary_repr(r)) for r in [x,cf,mf,f]]

# LZC($\mu$) using greycodes

In [ ]:
def _to_grey(num):
    return num ^ (num >> 1)
to_grey = np.frompyfunc(_to_grey, 1, 1)

def _from_grey(num):
    number = num;
    mask = number >> 1;
    while mask != 0:
        number = number ^ mask;
        mask = mask >> 1;
    return number

def mean_grey_lzc(m, o, fn=to_grey):
    dist = np.random.normal(m,o,size=1000)
    truth = to_u32(m)
    vals = to_u32v(dist)
    vals = fn(vals)
    xors = npxor(vals, fn(truth))
    lzcs = lzcv(xors).astype('uint8')
    return lzcs.mean()

def lzc_grey_arr(m, o, fn=to_grey):
    dist = np.random.normal(m,o,size=1000)
    truth = to_u32(m)
    vals = to_u32v(dist)
    vals = fn(vals)
    xors = npxor(vals, fn(truth))
    lzcs = lzcv(xors).astype('uint8')
    return lzcs

def lzc_arr(m, o):
    dist = np.random.normal(m,o,size=1000)
    truth = to_u32(m)
    vals = to_u32v(dist)
    xors = npxor(vals, truth)
    lzcs = lzcv(xors).astype('uint8')
    return lzcs

In [ ]:
mu = np.arange(1000)
sigma1 = mu*.20
sigma2 = mu*.10
sigma3 = mu*.05
sigma4 = mu*.01

result1_x_is_mu = [mean_grey_lzc(m,o) for m,o in zip(mu,sigma1)]
result2_x_is_mu = [mean_grey_lzc(m,o) for m,o in zip(mu,sigma2)]
result3_x_is_mu = [mean_grey_lzc(m,o) for m,o in zip(mu,sigma3)]
result4_x_is_mu = [mean_grey_lzc(m,o) for m,o in zip(mu,sigma4)]

factors = [1.75,1.625,1.5,1.25,1.125,1]
fcolors = ['crimson','magenta','skyblue','limegreen','olivedrab', 'goldenrod']

In [ ]:
# LZC(mu)
_, ax = plt.subplots(figsize=(15,5))
sns.lineplot(data=pd.DataFrame({'$\sigma = 0.20 \cdot \mu$':result1_x_is_mu, '$\sigma = 0.10 \cdot \mu$':result2_x_is_mu, 
                                '$\sigma = 0.05 \cdot \mu$':result3_x_is_mu, '$\sigma = 0.01 \cdot \mu$':result4_x_is_mu})[1:])
for x in range(4,10):
    flip = get_shifted_goal(2**x, method='c', mode='flip')
    mflip = get_shifted_goal(2**x, method='c', mode='mflip')
    cflip = get_shifted_goal(2**x, method='c', mode='cflip')
    _ = plt.axvline(2**x, alpha=.5, color=fcolors[x-4], ls=":")
    _ = plt.axvline(flip, alpha=.5, color=fcolors[x-4], ls="-")
    _ = plt.axvline(mflip, alpha=.5, color=fcolors[x-4], ls="-")
    _ = plt.axvline(cflip, alpha=.5, color=fcolors[x-4], ls="-")
plt.legend()
plt.title("LZC($\mu$) using grey codes")
ax.set_xlabel("$\mu$ of random normal distribution")
ax.set_ylabel("Average LZC count of distribution")
# plt.savefig('lzc_mu_set2.svg')
plt.show()

In [ ]:
for v in [2**x, cflip, mflip, flip]:
    print(v, np.binary_repr(v), to_grey(v), np.binary_repr(to_grey(v)))

In [ ]:
np.binary_repr(3 << 30, 32)

In [ ]:
np.binary_repr((4294967295 >> 5) + (3 << 30), 32)

In [ ]:
int("1100"*2, 2)

## LZC($\mu$) Analysis of potential manipulations of greycodes

In [ ]:
def to_grey_with_shift(val):
    temp = to_grey(val)
    return temp + 512

m, s = 512, 10
fn = to_grey_with_shift
dict(mglzc=mean_grey_lzc(get_close_flipping_value(m), s, fn).mean(), 
     mlzc=mean_lzc_group(get_close_flipping_value(m), s).mean()
)

In [ ]:
val = 512
for k in [val-1, val, val+1]:
    print(k, np.binary_repr(k), np.binary_repr(to_grey(k)))
print('')
for i in range(-10,10):
    k = val - i
    kn = k
    print("{:03} {:0>10} {:0>10}".format(k, np.binary_repr(kn), np.binary_repr(to_grey(kn))))

# LZC($\sigma$)

In [ ]:
sigma = np.arange(1000)
mu1 = 10 * sigma
mu2 = 50 * sigma
mu3 = 100 * sigma
mu4 = 500 * sigma

result1_x_is_sigma = [mean_lzc(m,o) for m,o in zip(mu1,sigma)]
result2_x_is_sigma = [mean_lzc(m,o) for m,o in zip(mu2,sigma)]
result3_x_is_sigma = [mean_lzc(m,o) for m,o in zip(mu3,sigma)]
result4_x_is_sigma = [mean_lzc(m,o) for m,o in zip(mu4,sigma)]

In [ ]:
# LZC(sigma)
_, ax = plt.subplots(figsize=(15,5))
df = pd.DataFrame({'$\mu =  10 \cdot \sigma$':result1_x_is_sigma, 
                   '$\mu =  50 \cdot \sigma$':result2_x_is_sigma, 
                   '$\mu = 100 \cdot \sigma$':result3_x_is_sigma, 
                   '$\mu = 500 \cdot \sigma$':result4_x_is_sigma,
})
sns.lineplot(data=df[3:])
for i,f in enumerate(factors):
    [plt.axvline(128*f, alpha=.5, color=fcolors[i], ls=":")];
plt.legend()
plt.title("LZC($\sigma$) using grey codes")
ax.set_xlabel("$\sigma$ of random normal distribution")
ax.set_ylabel("Average LZC count of distribution")
# plt.savefig('lzc_mu_set2.svg')
plt.show()

In [ ]:
sigma = np.arange(1000)
mu1 = [128 * 2] * sigma.size
mu2 = [128 * 1.5] * sigma.size
mu3 = [128 * 1.125] * sigma.size
mu4 = [128] * sigma.size

result1_x_is_sigma = [mean_lzc(m,o) for m,o in zip(mu1,sigma)]
result2_x_is_sigma = [mean_lzc(m,o) for m,o in zip(mu2,sigma)]
result3_x_is_sigma = [mean_lzc(m,o) for m,o in zip(mu3,sigma)]
result4_x_is_sigma = [mean_lzc(m,o) for m,o in zip(mu4,sigma)]

In [ ]:
# LZC(sigma)
_, ax = plt.subplots(figsize=(15,5))
df = pd.DataFrame({"$\mu = {}$".format(int(mu1[0])):result1_x_is_sigma, 
                   "$\mu = {}$".format(int(mu2[0])):result2_x_is_sigma, 
                   "$\mu = {}$".format(int(mu3[0])):result3_x_is_sigma, 
                   "$\mu = {}$".format(int(mu4[0])):result4_x_is_sigma,
})
sns.lineplot(data=df[2:])
plt.legend()
plt.title("LZC($\sigma$) using grey codes")
ax.set_xlabel("$\sigma$ of random normal distribution")
ax.set_ylabel("Average LZC count of distribution")
# plt.savefig('lzc_mu_set2.svg')
plt.show()

In [ ]:
# Vertical lines at above plot
base = 128
for i,f in enumerate(factors):
    print("{:.4f} {:>9} {:>10}".format(f, np.binary_repr(int(base*f), 8), fcolors[i]))

# $\sigma$ for different climate data

This part of the analysis must be specified more in detail. Espacially being specific about what should be analyzed. Options are:

- Mean/Std of residuals?
- Mean/Std of LZC of residuals?
- Mean/Std of Error of predictor?

In [ ]:
from glob import glob

In [ ]:
def describe_data(regex):
#     
    residual_files = glob(regex)
    df = pd.DataFrame(index=['mean', 'std'])

    for k in residual_files:
        data = np.fromfile(k, dtype='uint32')
        ps = lzcv(data)
        mean = ps.mean()
        std = ps.std()
        df[k[2:]] = [mean, std]

    df = df.T
    df['pcent'] = (df['std'] / df['mean']) * 100
    return df

def describe_lzc_data(regex):
    residual_files = sorted(glob(regex))
    df = pd.DataFrame(index=['mean', 'std'])

    for k in residual_files:
        data = np.fromfile(k, dtype='uint32')
        ps = ps.Series(data)
        mean = ps.mean()
        std = ps.std()
        df[k[2:]] = [mean, std]

    df = df.T
    df['pcent'] = (df['std'] / df['mean']) * 100
    return df

In [ ]:
# describe_data("./*.residual")

In [ ]:
# describe_data('../../pzip/data/*.raw')

In [ ]:
# describe_data("./*.bplanes")

In [ ]:
# describe_data("./*.nlzc")

In [ ]:
# df = describe_data('./*.residual')

In [ ]:
# df['mean'].plot()